In [4]:
import pandas as pd
import numpy as np
import re
import ftfy
import unidecode
from langdetect import detect
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.utils import shuffle

In [18]:
# Download NLTK dependencies
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...


True

In [8]:
paths = [
    "Dataset/completeSpamAssassin.csv",
    "Dataset/emails.csv",
    "Dataset/enronSpamSubset.csv",
    "Dataset/lingSpam.csv",
    "Dataset/spam mail.csv"
]
dfs = [pd.read_csv(p, encoding='utf-8') for p in paths]

In [10]:
for i, df in enumerate(dfs):
    unnamed = [c for c in df.columns if c.startswith("Unnamed")]
    if unnamed:
        df.drop(columns=unnamed, inplace=True)

In [11]:
rename_map = {
    "text": "Body",
    "Message": "Body",
    "Masseges": "Body",
    "Category": "Label",
    "spam": "Label",
    "label": "Label"
}

for i, df in enumerate(dfs):
    df.rename(columns={col: rename_map.get(col, col) for col in df.columns}, inplace=True)
    if "Label" in df.columns:
        df["Label"] = df["Label"].replace({"ham": 0, "spam": 1, "not spam": 0}).astype(int)

C:\Users\nikhi\AppData\Local\Temp\ipykernel_10060\1522610312.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Label"] = df["Label"].replace({"ham": 0, "spam": 1, "not spam": 0}).astype(int)


In [12]:
# Basic text cleaning
def basic_clean(text):
    text = str(text)
    text = ftfy.fix_text(text)
    text = unidecode.unidecode(text)
    text = re.sub(r"Subject:", "", text, flags=re.IGNORECASE)
    text = re.sub(r"http\S+|www\S+", " URL ", text)
    text = re.sub(r"\S+@\S+", " EMAIL ", text)
    text = re.sub(r"\d+", " NUMBER ", text)
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.lower()

for df in dfs:
    if "Body" in df.columns:
        df["Body"] = df["Body"].astype(str).apply(basic_clean)

In [13]:
# 5️⃣ Remove duplicates and empty entries
for i, df in enumerate(dfs):
    before = len(df)
    df.drop_duplicates(subset=["Body"], inplace=True)
    df.dropna(subset=["Body"], inplace=True)
    print(f"✅ Cleaned df{i+1}: removed {before - len(df)} duplicates/NaN")

✅ Cleaned df1: removed 906 duplicates/NaN
✅ Cleaned df2: removed 230 duplicates/NaN
✅ Cleaned df3: removed 617 duplicates/NaN
✅ Cleaned df4: removed 17 duplicates/NaN
✅ Cleaned df5: removed 476 duplicates/NaN


In [15]:
# 6️⃣ Merge datasets and shuffle
combined_df = pd.concat(dfs, ignore_index=True)
combined_df = combined_df.dropna(subset=["Body", "Label"])
combined_df = combined_df[combined_df["Body"].str.len() > 10]  # remove tiny texts
combined_df = shuffle(combined_df, random_state=42).reset_index(drop=True)
print(f"🧩 Combined dataset shape: {combined_df.shape}")

🧩 Combined dataset shape: (27648, 2)


In [16]:
# 7️⃣ Language filter (optional but useful)
def safe_detect(text):
    try:
        return detect(text)
    except:
        return "unknown"

combined_df["lang"] = combined_df["Body"].apply(safe_detect)
combined_df = combined_df[combined_df["lang"] == "en"]

print(f"🌍 English-only messages: {combined_df.shape}")

🌍 English-only messages: (26849, 3)


In [19]:
# 8️⃣ Lemmatization + stopword removal
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def advanced_clean(text):
    tokens = word_tokenize(text)
    cleaned = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word not in stop_words and len(word) > 2
    ]
    return " ".join(cleaned)

combined_df["Clean_Body"] = combined_df["Body"].apply(advanced_clean)

In [20]:
# 9️⃣ Handle class imbalance
spam_count = combined_df["Label"].sum()
ham_count = len(combined_df) - spam_count
print(f"📊 Spam: {spam_count}, Ham: {ham_count}")

# Option 1: Use undersampling if too imbalanced
if spam_count < ham_count / 3:
    ham_df = combined_df[combined_df["Label"] == 0].sample(spam_count * 2, random_state=42)
    spam_df = combined_df[combined_df["Label"] == 1]
    combined_df = pd.concat([ham_df, spam_df]).sample(frac=1, random_state=42).reset_index(drop=True)

print(f"✅ Final balanced shape: {combined_df.shape}")

📊 Spam: 8217, Ham: 18632
✅ Final balanced shape: (26849, 4)


In [21]:
# 🔚 Save the cleaned dataset
combined_df.to_csv("Dataset/cleaned_spam_dataset.csv", index=False)
print("💾 Saved cleaned dataset → Dataset/cleaned_spam_dataset.csv")

💾 Saved cleaned dataset → Dataset/cleaned_spam_dataset.csv
